In [1]:
%cd /root/GitHub/Integrations/

/root/GitHub/Integrations


In [3]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## Downloading the dataset<a name="Downloading"></a>
Download the [direct marketing dataset](!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) from the sample data s3 bucket. 

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [4]:
!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
!unzip -o bank-additional.zip

local_data_path = './bank-additional/bank-additional-full.csv'


--2020-03-09 22:56:48--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.221.233
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.221.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

bank-additional.zip 100%[===================>] 422.68K  1.47MB/s    in 0.3s    

2020-03-09 22:56:49 (1.47 MB/s) - ‘bank-additional.zip’ saved [432828/432828]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


## Upload the dataset to Amazon S3<a name="Uploading"></a>

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.


Read the data into a Pandas data frame and take a look.

In [5]:
import pandas as pd

data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


Note that there are 20 features to help predict the target column 'y'.

Amazon SageMaker Autopilot takes care of preprocessing your data for you. You do not need to perform conventional data preprocssing techniques such as handling missing values, converting categorical features to numeric features, scaling data, and handling more complicated data types.

Moreover, splitting the dataset into training and validation splits is not necessary. Autopilot takes care of this for you. You may, however, want to split out a test set. That's next, although you use it for batch inference at the end instead of testing the model.


### Reserve some data for calling batch inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [6]:
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['y'])

### Upload the dataset to Amazon S3
Copy the file to Amazon Simple Storage Service (Amazon S3) in a .csv format for Amazon SageMaker training to use.

In [7]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-2-308412838853/sagemaker/autopilot-dm/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-east-2-308412838853/sagemaker/autopilot-dm/test/test_data.csv


## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking a Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the column of the dataset you want to predict (`y` in this case) 
* An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order, is expected to have a header row.

In [8]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'y'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

## Launching the SageMaker Autopilot Job<a name="Launching"></a>

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. 

In [9]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-banking-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      RoleArn=role)

AutoMLJobName: automl-banking-09-22-57-10


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-2:308412838853:automl-job/automl-banking-09-22-57-10',
 'ResponseMetadata': {'RequestId': '392bde85-f041-4d1e-a6f4-c5d5c854420d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '392bde85-f041-4d1e-a6f4-c5d5c854420d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Mon, 09 Mar 2020 22:57:10 GMT'},
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress<a name="Tracking"></a>
SageMaker Autopilot job consists of the following high-level steps : 
* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [10]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEng

## Results

Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job. 

In [11]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-cd94bd35d15c4e38a7-060-b20510cd', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy', 'Value': 0.9157689809799194}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:308412838853:processing-job/db-1-85086775aca547d5881363100de5da921a28db2d8943428d940390cd7d', 'CandidateStepName': 'db-1-85086775aca547d5881363100de5da921a28db2d8943428d940390cd7d'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:308412838853:training-job/automl-ban-dpp9-1-d60bddc33f8b4670851e9a7188c484ea284a3563f7934', 'CandidateStepName': 'automl-ban-dpp9-1-d60bddc33f8b4670851e9a7188c484ea284a3563f7934'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:308412838853:transform-job/automl-ban-dpp9-csv-1-17fae5d2c4764a13be5a29021fae0c9f94e0f6801', 'CandidateStepNa

### Perform batch inference using the best candidate

Now that you have successfully completed the SageMaker Autopilot job on the dataset, create a model from any of the candidates by using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). 

In [12]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

model_name = 'automl-banking-model-' + timestamp_suffix

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-east-2:308412838853:model/automl-banking-model-10-05-42-38


In [13]:
best_candidate['InferenceContainers'][1]['ModelDataUrl']

's3://sagemaker-us-east-2-308412838853/sagemaker/autopilot-dm/output/automl-banking-09-22-57-10/tuning/automl-ban-dpp9-xgb/tuning-job-1-cd94bd35d15c4e38a7-060-b20510cd/output/model.tar.gz'

You can use batch inference by using Amazon SageMaker batch transform. The same model can also be deployed to perform online inference using Amazon SageMaker hosting.

In [14]:
transform_job_name = 'automl-banking-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-2:308412838853:transform-job/automl-banking-transform-10-05-42-38',
 'ResponseMetadata': {'RequestId': 'faa9e1eb-f92d-42d2-8f59-7fd4dcd29df5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'faa9e1eb-f92d-42d2-8f59-7fd4dcd29df5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Tue, 10 Mar 2020 05:42:43 GMT'},
  'RetryAttempts': 0}}

Watch the transform job for completion.

In [15]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


Now let's view the results of the transform job:

In [16]:
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,no
0,no
1,no
2,no
3,no
4,no
...,...
8232,yes
8233,yes
8234,no
8235,yes


### View other candidates explored by SageMaker Autopilot
You can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by SageMaker Autopilot and sort them by their final performance metric.

In [17]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

1  tuning-job-1-cd94bd35d15c4e38a7-060-b20510cd  0.9157689809799194
2  tuning-job-1-cd94bd35d15c4e38a7-121-02efc977  0.9150099754333496
3  tuning-job-1-cd94bd35d15c4e38a7-230-82fe5b01  0.914555013179779
4  tuning-job-1-cd94bd35d15c4e38a7-243-8f89de91  0.9142510294914246
5  tuning-job-1-cd94bd35d15c4e38a7-171-0f3acce7  0.9140989780426025
6  tuning-job-1-cd94bd35d15c4e38a7-195-e53c31ee  0.9137960076332092
7  tuning-job-1-cd94bd35d15c4e38a7-063-48772597  0.913644015789032
8  tuning-job-1-cd94bd35d15c4e38a7-087-53ab8df9  0.913644015789032
9  tuning-job-1-cd94bd35d15c4e38a7-104-838958b0  0.9134920239448547
10  tuning-job-1-cd94bd35d15c4e38a7-201-4c0dfcdc  0.9134920239448547


### Candidate Generation Notebook
    
Sagemaker AutoPilot also auto-generates a Candidate Definitions notebook. This notebook can be used to interactively step through the various steps taken by the Sagemaker Autopilot to arrive at the best candidate. This notebook can also be used to override various runtime parameters like parallelism, hardware used, algorithms explored, feature extraction scripts and more.
    
The notebook can be downloaded from the following Amazon S3 location:

In [18]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']

!aws s3 cp s3://sagemaker-us-east-2-023375022819/sagemaker/autopilot-dm/output/automl-banking-27-17-05-08/sagemaker-automl-candidates/pr-1-6dfa0899a9d54be686b3fc404fe27585a8ffe49b4ea7413595fa18ee6b/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb .

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


### Data Exploration Notebook
Sagemaker Autopilot also auto-generates a Data Exploration notebook, which can be downloaded from the following Amazon S3 location:

In [20]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
!aws s3 cp s3://sagemaker-us-east-2-023375022819/sagemaker/autopilot-dm/output/automl-banking-27-17-05-08/sagemaker-automl-candidates/pr-1-6dfa0899a9d54be686b3fc404fe27585a8ffe49b4ea7413595fa18ee6b/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb .

fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden


## Feature Importance

Now, let's inspect the trained XGBoost model and see what features it found to be the most informative in predicting the output.

In [21]:
import os
os.system('aws s3 cp {} .'.format(best_candidate['InferenceContainers'][1]['ModelDataUrl'])) 

0

In [22]:
!tar -xvzf model.tar.gz

xgboost-model


In [23]:
!pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 48 kB/s s eta 0:00:01   |██▌                             | 11.1 MB 2.8 MB/s eta 0:00:47


In [24]:
import pickle as pkl
import xgboost
booster = pkl.load(open('xgboost-model', 'rb'))
booster.get_score()
booster.get_fscore()

{'f9': 12,
 'f1': 101,
 'f3': 11,
 'f7': 8,
 'f8': 33,
 'f51': 4,
 'f41': 2,
 'f59': 1,
 'f5': 19,
 'f55': 2,
 'f0': 1,
 'f43': 3,
 'f6': 14,
 'f49': 2,
 'f32': 2,
 'f44': 1,
 'f60': 2,
 'f57': 2,
 'f50': 1}

In [31]:
import sys

!{sys.executable} -m pip install sagemaker-experiments sagemaker

  Using cached sagemaker_experiments-0.1.8-py3-none-any.whl (26 kB)


In [32]:
search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

In [39]:
import sagemaker
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

automl_experiment = Experiment.load(experiment_name='automl-banking-09-22-57-10-aws-auto-ml-job')


trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=automl_experiment.experiment_name
)

In [40]:
analytic_table = trial_component_analytics.dataframe()

In [44]:
analytic_table.shape

(271, 120)

In [45]:
analytic_table

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,_tuning_objective_metric,alpha,colsample_bytree,eta,gamma,lambda,max_depth,min_child_weight,num_round,objective,subsample,ObjectiveMetric - Min,ObjectiveMetric - Max,ObjectiveMetric - Avg,ObjectiveMetric - StdDev,ObjectiveMetric - Last,ObjectiveMetric - Count,validation:error - Min,validation:error - Max,validation:error - Avg,validation:error - StdDev,validation:error - Last,validation:error - Count,validation:accuracy - Min,validation:accuracy - Max,validation:accuracy - Avg,validation:accuracy - StdDev,validation:accuracy - Last,validation:accuracy - Count,train:error - Min,train:error - Max,train:error - Avg,train:error - StdDev,train:error - Last,train:error - Count,train:accuracy - Min,train:accuracy - Max,train:accuracy - Avg,train:accuracy - StdDev,train:accuracy - Last,train:accuracy - Count,binary_classifier_model_selection_criteria,l1,learning_rate,loss,mini_batch_size,num_models,positive_example_weight_mult,predictor_type,wd,validation:objective_loss - Min,validation:objective_loss - Max,validation:objective_loss - Avg,validation:objective_loss - StdDev,validation:objective_loss - Last,validation:objective_loss - Count,train:progress - Min,train:progress - Max,train:progress - Avg,train:progress - StdDev,train:progress - Last,train:progress - Count,train:throughput - Min,train:throughput - Max,train:throughput - Avg,train:throughput - StdDev,train:throughput - Last,train:throughput - Count,validation:recall - Min,validation:recall - Max,validation:recall - Avg,validation:recall - StdDev,validation:recall - Last,validation:recall - Count,validation:binary_classification_accuracy - Min,validation:binary_classification_accuracy - Max,validation:binary_classification_accuracy - Avg,validation:binary_classification_accuracy - StdDev,validation:binary_classification_accuracy - Last,validation:binary_classification_accuracy - Count,train:objective_loss - Min,train:objective_loss - Max,train:objective_loss - Avg,train:objective_loss - StdDev,train:objective_loss - Last,train:objective_loss - Count,validation:objective_loss:final - Min,validation:objective_loss:final - Max,validation:objective_loss:final - Avg,validation:objective_loss:final - StdDev,validation:objective_loss:final - Last,validation:objective_loss:final - Count,validation:binary_f_beta - Min,validation:binary_f_beta - Max,validation:binary_f_beta - Avg,validation:binary_f_beta - StdDev,validation:binary_f_beta - Last,validation:binary_f_beta - Count,validation:precision - Min,validation:precision - Max,validation:precision - Avg,validation:precision - StdDev,validation:precision - Last,validation:precision - Count,SageMaker.ModelName,SageMaker.ModelPrimary.DataUrl,SageMaker.ModelPrimary.Image,processor_module,sagemaker_program,sagemaker_submit_directory,input_channel_mode,job_name,label_col
0,tuning-job-1-cd94bd35d15c4e38a7-250-c01b2ff8-a...,tuning-job-1-cd94bd35d15c4e38a7-250-c01b2ff8-a...,arn:aws:sagemaker:us-east-2:308412838853:train...,257758044811.dkr.ecr.us-east-2.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,0.080787,0.592471,0.590130,2.335417,0.006172,28.0,0.024237,512.0,binary:hinge,0.950908,0.000000,0.000000,0.000000,0.000000,0.892548,0.0,0.000000,0.000000,0.000000,0.000000,0.107452,0.0,0.000000,0.000000,0.000000,0.000000,0.892548,0.0,0.000000,0.000000,0.000000,0.000000,0.000038,0.0,0.000000,0.000000,0.000000,0.000000,0.999962,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tuning-job-1-cd94bd35d15c4e38a7-249-db7fd3c8-a...,tuning-job-1-cd94bd35d15c4e38a7-249-db7fd3c8-a...,arn:aws:sagemaker:us-east-2:308412838853:train...,257758044811.dkr.ecr.us-east-2.amazonaw